<a href="https://colab.research.google.com/github/mirklys/little-projects/blob/main/thesis/testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Apr 20 14:36:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip3 install Box2D
!pip3 install box2d-py
!pip3 install gym[all]
!pip3 install gym[Box_2D]

In [ ]:
!pip install stable_baselines
!pip install stable_baselines3

In [5]:
!pip install tensorflow

In [6]:
import gym
import os
import numpy as np
import torch as th
from torch import nn
from torch.distributions.bernoulli import Bernoulli
import matplotlib.pyplot as plt
from collections import defaultdict
import time

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.utils import get_device
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines.common import set_global_seeds, make_vec_env
from stable_baselines3.common.monitor import Monitor

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [8]:
device = th.device('cuda' if th.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [9]:
PATH_BASE = '/content/gdrive/MyDrive/Thesis Project'
PATH_DATA = os.path.join(PATH_BASE, 'data/')
PATH_NETWORKS = os.path.join(PATH_BASE, 'networks/')
PATH_PLOTS = os.path.join(PATH_BASE, 'plots/')
PATH_RESULTS = os.path.join(PATH_BASE, 'results/')
PATH_LOGS = os.path.join(PATH_BASE, 'logs/')
os.chdir(PATH_BASE)

In [ ]:
def test_network(game:str, size:int, dropout:float) -> str:
    title = "{}.{}x{}.dropout_{}".format(game, size, size, dropout)
    env = make_vec_env(game, n_envs=10, seed=0, vec_env_cls=DummyVecEnv)
    model = PPO.load(os.path.join(PATH_NETWORKS, game, title))

    rew, std = evaluate_policy(model, env, n_eval_episodes=10)

    return "Networks cumulative reward {:.2f} ±{:.2f}".format(rew, std)

In [ ]:
test_network('BipedalWalker-v3', 128, 0.0)

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


'Networks cumulative reward 317.78 ±0.60'

In [ ]:
test_network('BipedalWalker-v3', 128, 0.2)

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


'Networks cumulative reward 200.50 ±135.86'

In [10]:
def parse_title(title: str) -> tuple:
    splitted = title.split(".")
    game = splitted[0]
    size = splitted[1].split("x")[0]
    dropout = float(title.split("_")[1])
    return game, size, dropout

In [11]:
def eval_with_mask(model, percent_to_mask, env):
    rewards = []
    for i in range(20): # repeat over multiple random masks)
        model.policy.features_extractor.mask_units(percent_to_mask=percent_to_mask)
        mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)
        rewards.append(mean_reward)
    return np.round(np.mean(rewards), 3), np.round(np.std(rewards), 3)

In [ ]:
import time
import warnings
warnings.filterwarnings("ignore")

inc = 0.05
nets = os.listdir(os.path.join(PATH_NETWORKS, 'CartPole-v1'))
evals = defaultdict(list)
env = make_vec_env('CartPole-v1', n_envs=10, seed=0, vec_env_cls=DummyVecEnv)
for net in nets:
    print(net)
    game, size, dropout = parse_title(net)
    model = PPO.load(os.path.join(PATH_NETWORKS, game, net), device=device)
    model.set_env(env)
    model.policy.features_extractor.training = False
    print("Masking size: ")
    mean_rews, std_rews = [], []
    for mask in np.arange(0., 1.+inc, inc):
        print(mask, end=" ")
        eval = eval_with_mask(model, percent_to_mask=float(mask), env=env)
        print(eval, end='\n')
        mean_rews.append(eval[0])
        std_rews.append(eval[1])
    print('\n')
    evals[(game, size, dropout)].append((mean_rews, std_rews))


CartPole-v1.128x128.dropout_0.0
Masking size: 
0.0 (500.0, 0.0)
0.05 (500.0, 0.0)
0.1 (500.0, 0.0)
0.15000000000000002 (500.0, 0.0)
0.2 (500.0, 0.0)
0.25 (500.0, 0.0)
0.30000000000000004 (500.0, 0.0)
0.35000000000000003 (500.0, 0.0)
0.4 (500.0, 0.0)
0.45 (500.0, 0.0)
0.5 (500.0, 0.0)
0.55 (500.0, 0.0)
0.6000000000000001 (500.0, 0.0)
0.65 (500.0, 0.0)
0.7000000000000001 (500.0, 0.0)
0.75 (500.0, 0.0)
0.8 (500.0, 0.0)
0.8500000000000001 (480.374, 85.548)
0.9 (474.385, 65.125)
0.9500000000000001 (403.878, 147.917)
1.0 (9.381, 0.068)


CartPole-v1.128x128.dropout_0.2
Masking size: 
0.0 (500.0, 0.0)
0.05 (500.0, 0.0)
0.1 (500.0, 0.0)
0.15000000000000002 (500.0, 0.0)
0.2 (500.0, 0.0)
0.25 (500.0, 0.0)
0.30000000000000004 (500.0, 0.0)
0.35000000000000003 (500.0, 0.0)
0.4 (500.0, 0.0)
0.45 (500.0, 0.0)
0.5 (500.0, 0.0)
0.55 (500.0, 0.0)
0.6000000000000001 (500.0, 0.0)
0.65 (500.0, 0.0)
0.7000000000000001 (500.0, 0.0)
0.75 (500.0, 0.0)
0.8 (500.0, 0.0)
0.8500000000000001 (500.0, 0.0)
0.9 (500.0

In [ ]:
import pickle

with open(os.path.join(PATH_RESULTS, "evaluations_cartpole.pickle"), "wb") as f:
    pickle.dump(evals, f)

In [ ]:
def generate_net_name(game, size, dropout):
    return "{}.{}x{}.dropout_{}".format(game, size, size, dropout)

In [ ]:
num_training_steps = 1000
overtrained_evals = defaultdict(list)
for net, eval in evals.items():
    game, size, dropout = net
    masks = list(np.arange(0., 1.+inc, inc))
    for i, e in enumerate(eval):
        rew, std = e
        if rew < 500:
            name = generate_net_name(game, size, dropout)
            model = PPO.load(os.path.join(PATH_NETWORKS, game, name), device=device)
            mask = masks[i]
            model.set_env(env)
            model.policy.features_extractor.job = 'train_masked'
            model.policy.features_extractor.percent_to_mask = mask
            model.policy.features_extractor.mask_units(percent_to_mask=mask)

            total_further_train_steps = 0

            print(name)
            print("steps: ", total_further_train_steps, ". rew: ", rew, "+- ", std)
            while rew < 500:
                model.learn(num_training_steps)
                rew, std = evaluate_policy(model, env, n_eval_episodes=100)
                total_further_train_steps = num_training_steps
                print("steps: ", total_further_train_steps, ". rew: ", rew, "+- ", std)
                if std/rew < 0.05: break
            
            name += '.mask_{}.overtrained'.format(mask)
            overtrained_evals[name].append((rew, std, total_further_train_steps))


In [ ]:
with open(os.path.join(PATH_RESULTS, "evaluations_overtrained_cartpole.pickle"), "wb") as f:
    pickle.dump(overtrained_evals, f)